<a href="https://colab.research.google.com/github/llimental/ai-security/blob/master/First%20Assignment/LeeSangyun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 하단의 코드는 yunjey의 pytorch-tutorial을 바탕으로 주석 해석한 것임을 밝힘 #
# 출처: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py #
# 인공지능과 보안기술 강의 과제용으로 튜토리얼 코드를 가져와 영문 주석 번역 및 모르는 부분 추가하였음 #

import torch  # torch import
import torchvision # cnn(convolutional neural network)을 위한 torchvision import
import torch.nn as nn # nn(신경망)기능 import
import numpy as np # numpy 모듈 np라는 이름으로 import
import torchvision.transforms as transforms # 이미지 변환을 위해 torchvision.transforms 기능을 transforms 라는 이름으로 import


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# tensor 만들기
x = torch.tensor(1., requires_grad=True)
# 기울기를 구하기 위해 require_grad 값을 True로 줌
# torch.tensor는 값 복사를 사용하여 새로운 텐서 자료형 인스턴스 생성
# 즉, 1. 값 복사하여 기울기 여부 True로 x에 텐서를 하나 생성하는 것임
w = torch.tensor(2., requires_grad=True)
# w와 b역시 x와 마찬가지로 각각의 값을 복사하여 기울기 True로 텐서를 생성
b = torch.tensor(3., requires_grad=True)

# 계산 그래프 작성
y = w * x + b    # y = 2 * x + 3 -> 위에서 만든 텐서의 값

# 경사도 계산
y.backward() # backward = gradient(변화도) 자동계산
# autograd를 사용하여 역전파 단계 계산

# 경사도 출력
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# (10, 3)과 (10, 2)의 모양에 랜덤한 수를 갖는 tensor 두 개를 생성
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# fully connected layer 만들기
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
# Loss function(손실함수): 모델을 통해 생성된 결과 값과 실제로 발생하기를 원했던 값간의 차이를 계산하는 함수.
criterion = nn.MSELoss() # 간단한 손실함수. 입력과 정답 사이의 평균 제곱 오차를 계산
# 최적화를 위해 SGD(Stochastic Gradient Descent, 확률적 경사하강법)사용
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
# -> 손실 함수의 값을 계산하기 위한 것
pred = linear(x)

# Compute loss.
# 손실 계산
loss = criterion(pred, y)
print('loss: ', loss.item()) # 손실값 출력

# Backward pass.
# -> learnable parameters의 gradients를 계산
loss.backward()

# 경사도 출력
# 하단의 weight.grad와 bias.grad에 대한 코드는 아무리 찾아도 위의 경사도 출력만 나오고
# 해설이 없어 가중치 기울기 및 편향 기울기를 표현하는 것으로 이해하였다.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
# 1단계 경사 강하
optimizer.step()

# 낮은 레벨에서의 경사 하강도 또한 수행할 수 있음
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# 1단계 경사 강하 후 손실 출력
pred = linear(x)
loss = criterion(pred, y) # 손실 계산 파트
print('loss after 1 step optimization: ', loss.item()) # 경사 강하 후 값을 출력


# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# numpy 배열 생성
# numpy는 tensor와 동기화 된다고 적혀있었음(참고)
x = np.array([[1, 2], [3, 4]])

# 생성된 numpy 배열을 torch tensor로 변환
y = torch.from_numpy(x)

# torch tensor를 numpy 배열로 변환
z = y.numpy()


# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# CIFAR-10 데이터셋을 다운로드하고 구성
# CIFAR-10이란 기계학습에서 흔히 사용되는 벤치마크 문제.
# RGB 32x32 픽셀 이미지를 10개 카테고리로 분류하는 것이 목표
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# 데이터 쌍을 하나씩 가져온다(디스크에서 데이터를 읽어옴_CIFAR-10)
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (매우 간단한 방법으로 queue와 thread를 제공).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# 반복이 시작되면, 큐와 쓰레드가 파일에서 데이터를 불러오기 시작
data_iter = iter(train_loader)

# Mini-batch 이미지와 라벨
images, labels = data_iter.next()

# data loader의 실제 사용은 아래와 같음
for images, labels in train_loader:
    # 훈련 코드를 여기에 작성해야 함
    pass


# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

# 각 개인의 커스텀 데이터셋을 아래에 작성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. 파일 경로 또는 파일 이름의 리스트를 초기화
        pass
    def __getitem__(self, index):
        # TODO
        # 1. 파일에서 데이터 하나를 읽어옴(예. numpy.fromfile, PIL.Image.open을 사용)
        # 2. 데이터 사전처리(예. torchvision.Transform)
        # 3. 데이터 쌍 반환(예. 이미지와 라벨)
        pass
    def __len__(self):
        # 0을 당산의 데이터셋의 총 사이즈로 바꿔야함
        return 0 

# 그런 다음 미리 구축된 data loader를 사용할 수 있음
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# 미리 훈련된 ResNet-18을 다운로드하고 불러옴
# ResNet-18은 ILSVRC에서 2015년 우승한 심층 신경망 모델.
resnet = torchvision.models.resnet18(pretrained=True)

# 모델의 상단 레이어만 미세 조정하길 원한다면, 아래와 같이 설정
# Fine-tuning이란, 모델의 파라미터를 미세하게 조정하는 행위.
# 특히 딥러닝에서는 이미 존재하는 모델에 추가 데이터를 투입하여 파라미터를 업데이트 하는 것을 말함
for param in resnet.parameters():
    param.requires_grad = False

# 미세 조정을 위해 상단 레이어를 교체
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 = 예시

# Forward pass(손실함수의 값 계산).
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# 전체 모델을 저장하고 불러오기
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# 모델 파라미터만 저장하고 불러오기(권장됨).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

In [0]:
# 하단의 코드는 yunjey의 pytorch-tutorial을 바탕으로 주석 해석한 것임을 밝힘 #
# 출처: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py #
# 인공지능과 보안기술 강의 과제용으로 튜토리얼 코드를 가져와 영문 주석 번역 및 모르는 부분 추가하였음 #
# 튜토리얼 코드임을 감안, 추후 개인프로젝트를 위해서라도 과제 제출용 주석 코드를 한 개의 main.py가 아니라 tutorials/01-basics 내 여러 main.py를 가지고 작업함 #

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# 기기 설정
# cuda 코드가 있는데, 이는 GPU가 있는 데스크탑 또는 노트북에서 실행할 시 사용하며 이외에는 'cpu'를 사용함을 알 수 있다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
# -> 머신러닝 학습을 할 때에 더 효과가 좋도록 하는 주 변수가 아닌 자동 설정 되는 변수를 의미
# 출처: 머신러닝에서의 hyperparameter - MATH & ML 배우기(https://blogyong.tistory.com/8)
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST 데이터셋
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)
# root: MNIST/processed/training.pt와 MNIST/processed/test.pt가 있는 데이터셋의 루트 디렉토리
# train: bool형식의 매개를 가지며, optional임. True라면 training.pt로부터 데이터셋을 생성
# train Ct. True 이외의 것이라면 test.pt로부터 데이터셋을 생성

# transform(callable, optional): PIL 이미지를 가져와서 변환된 버전을 반환하는 기능/변환
# download(bool, optional): true라면 인터넷으로부터 데이터셋을 다운받아 루트 디렉토리에 넣는다.
# download Ct. 만약 데이터셋이 이미 다운로드되어 있다면 다시 다운로드 받지 않는다.

# 즉 위의 코드는 루트 디렉토리를 지정해주고 있으며, train이 True이기 때문에 training.pt에서 데이터셋을 생성하였고
# PIL 이미지를 변환하여 텐서로 반환하며, 데이터셋은 인터넷으로부터 다운로드 받는것임을 추측할 수 있다.

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())
# 위의 코드와 반대로, train이 False이므로 test.pt로부터 데이터셋을 생성했음을 알 수 있다.

# 데이터 로더
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# dataset: 불러올 데이터. 여기서는 train_dataset 이므로 아까 MNIST 데이터셋을 활용한 train_dataset을 사용
# batch_size(int, optional): 매 batch마다 얼만큼의 샘플을 불러올 것인가(기본값: 1)
# 여기서 batch란, 내부 모델 파라미터를 업데이트 하기 전 샘플의 개수 의미
# shuffle(bool, optional): 매 epoch마다 다시 섞으려면 True이며 기본값은 False. 여기서는 True이기 때문에
# epoch: 알고리즘이 전체 훈련 데이터셋을 반복해서 학습하는 횟수.
# epoch ct. 1개의 epoch는 훈련데이터셋의 각 샘플이 내부모델 파라미터들을 업데이트하는데 사용될 수 있음을 의미
# ex. 한 개의 batch를 가지고 있는 epoch는 'batch gradient descent' 알고리즘으로 불림
# epoch의 크기는 보통 큰 값. 흔히 수백 혹은 수천의 값(에러를 충분히 최소화하기 위해)
# 출처: 신경망의 핵심 parameter인 batch 그리고 epoch(https://blog.naver.com/tjdudwo93/221325685474)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# 한 개의 은닉층을 갖고 있는 완전히 연결된 신경망(Fully connected neural network)
# class = 여러 개의 함수정의(def)를 묶는 것
class NeuralNet(nn.Module):
    # init = 초기화, class 사용시 처음 실행되는 것
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# 손실(Loss)와 최적화(Optimizer)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# 모델 훈련
total_step = len(train_loader) # 총 단계는 훈련 로더의 len(글자 수)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # 텐서를 설정된 기기로 이동
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        # 이 때에는 각 필터를 입력 볼륨의 가로/세로 차원으로 슬라이딩 시키며(정확히는 convolve 시키며)
        # 2차원의 액티베이션 맵을 생성. 필터를 입력 위로 슬라이딩 시킬 때, 필터와 입력의 요소들 사이의 내적 연산이 이뤄짐.
        # 출처: 컨볼루션 신경망(http://aikorea.org/cs231n/convolutional-networks/)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        # Backward란 계산이 완료된 후 .backward()를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다.
        optimizer.zero_grad() # 이전 epoch에서 계산되어 있는 parameter의 gradient를 0으로 초기화
        loss.backward() # loss와 chain rule을 활용하여 모델의 각 레이어에서 gradient를 계산
        optimizer.step() # w ← w − αΔw 식에 의해 모델의 parameter를 update
        # 위의 설명 출처: PyTorch 사용법 - 03. How to Use PyTorch(https://greeksharifa.github.io/pytorch/2018/11/10/pytorch-usage-03-How-to-Use-PyTorch/#train-model)
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# 모델 테스트
# 테스트 단계에서, 변화도(gradients)를 계산할 필요 없음(메모리 효율성을 위해)
with torch.no_grad(): # 변화도 계산 x를 위해 torch.grad()가 아니라 torch.no_grad() 사용
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# 모델의 체크포인트 저장
torch.save(model.state_dict(), 'model.ckpt')

In [0]:
# 하단의 코드는 yunjey의 pytorch-tutorial을 바탕으로 주석 해석한 것임을 밝힘 #
# 출처: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/linear_regression/main.py #
# 인공지능과 보안기술 강의 과제용으로 튜토리얼 코드를 가져와 영문 주석 번역 및 모르는 부분 추가하였음 #
# 튜토리얼 코드임을 감안, 추후 개인프로젝트를 위해서라도 과제 제출용 주석 코드를 한 개의 main.py가 아니라 tutorials/01-basics 내 여러 main.py를 가지고 작업함 #

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt # Matplotlib이란 파이썬에서 자료를 차트나 플롯으로 시각화하는 패키지
# 그 중 pyplot 모듈은 2D 그래프와 차트를 구성하는 것과 관련된 많은 기능을 가짐.
# 해당 모듈의 기능에는 plot, show, title, xlable, ylabel등이 있음


# Hyper-parameters
# 1_2 주석과 마찬가지로, Hyper-parameters란 자동 설정 되는 변수를 의미
input_size = 1 # 입력 차원(dimension)
output_size = 1 # 출력 차원(dimentsion)
num_epochs = 60 # 60번 함수 실행
learning_rate = 0.001 # 학습률(learning_rate). 한 번 학습할 때 얼마만큼 학습해야 하는지 학습 양을 의미

# Toy 데이터셋
x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
                    [9.779], [6.182], [7.59], [2.167], [7.042], 
                    [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
                    [3.366], [2.596], [2.53], [1.221], [2.827], 
                    [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)

# 선형 회귀(Linear regression) 모델
# 통계학에서, 선형 회귀는 종속 변수 y와 한 개 이상의 독립 변수 x와의 선형 상관 관계를 모델링하는 회귀분석 기법(출처: 위키백과)
model = nn.Linear(input_size, output_size)

# 손실(Loss)와 최적화(optimizer)
criterion = nn.MSELoss()
# MSELoss()는 Mean Squared Error Loss의 줄임말로, 원본과 예측의 차이의 제곱의 평균을 구해준다는 의미
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# model.parameters()를 통해 모델의 파라미터들을 optimizer에 전달하면
# 이전에 변화도(gradient)를 사용하여 업데이트하던 w = w - grad * learning rate 같은 식을 자동으로 진행
# 단순히 빼는 것이 아니라 SGD(Stochastic Gradient Descent)라는 방법을 써서 optimizing 진행
# lr 값으로 learning_rate를 정해줄 수 있지만 여기서는 앞서 learning_rate 변수에 0.001을 집어넣었기 때문에 최적화에 넘어가는 lr은 0.001
# 출처: Pytorch 머신러닝 튜토리얼 강의 5(https://wingnim.tistory.com/30)

# 모델 훈련
for epoch in range(num_epochs):
    # numpy 배열을 torch 텐서로 변환
    inputs = torch.from_numpy(x_train) # ndarray 객체를 받아 값 참조(reference)를 사용하여 텐서 자료형 뷰(view)를 만듦
    targets = torch.from_numpy(y_train)

    # Forward pass
    # 손실함수 값 계산
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# 그래프를 그림
predicted = model(torch.from_numpy(x_train)).detach().numpy()
plt.plot(x_train, y_train, 'ro', label='Original data') # plot 데이터에 해당 매개변수 사용
plt.plot(x_train, predicted, label='Fitted line')
plt.legend()
plt.show() # 윈도우에 데이터 표시

# 모델 체크포인트 저장
torch.save(model.state_dict(), 'model.ckpt')